In [197]:
from jinja2 import Environment, FileSystemLoader
import numpy as np
import pandas as pd 

In [198]:
opt_f='bromide.log'
not_opt_f='coumarin.log'
with open(opt_f, 'r') as f:
    lines = f.readlines()

In [199]:
# Check if optimization was succesfull 
opt = False
for line in lines:
    if  "**** THE GEOMETRY SEARCH IS NOT CONVERGED! ****" in line:
        opt = True

In [200]:
# If optimization is successfull 
if not opt:
    data = [] 
    for num, line in enumerate(lines):
        if "TOTAL NUMBER OF ATOMS" in line:
            n_atoms = int(line.split()[5])
        if "EQUILIBRIUM" in line:
            start_coord_index = num + 4
            end_coord_index = start_coord_index + n_atoms
            for i in range(start_coord_index, end_coord_index):
                data.append(lines[i].split())
    atomic_number = []
    element_symbol = []
    x_coord = []
    y_coord = []
    z_coord = []
    for line in data:
        element_symbol.append(line[0])
        atomic_number.append(line[1])
        x_coord.append(line[2])
        y_coord.append(line[3])
        z_coord.append(line[4])
        
    df = pd.DataFrame([element_symbol, x_coord, y_coord, z_coord])
    df = df.transpose()
    df[0] = df[0].map("{0:<4s}".format)
    df[1] = pd.to_numeric(df[1], downcast="float").map("{0:>16.10f}".format)
    df[2] = pd.to_numeric(df[2], downcast="float").map("{0:>16.10f}".format)
    df[3] = pd.to_numeric(df[3], downcast="float").map("{0:>16.10f}".format)
    
    result = df[0] + df[1] + df[2] + df[3]

In [201]:
file_loader=FileSystemLoader("../templates")
environment = Environment(loader=file_loader)
if not opt:
    template = environment.get_template("structure_input_template.xyz")

    MOLNAME="bromide"
    content = template.render(N_ATOMS=n_atoms, MOLNAME=MOLNAME, COORDINATES=result)
    print(content)
    
    # write into a file
    # filename = MOLNAME+'.xyz'
    # with open(filename, mode="w", encoding="utf-8") as message:
    #    message.write(content)

18
bromide
BR     -0.0933869332    2.8189375401    0.3264436722
C       0.0379065201    0.9059296846    0.1106576398
C       1.2129669189    0.3364498615   -0.3630618751
C      -1.0413632393    0.0920419991    0.4298464358
BR      2.7191379070    1.4585579634   -0.8008101583
C       1.2956054211   -1.0396865606   -0.5205134749
BR     -2.6734335423    0.8742509484    1.0967696905
C      -0.9391810894   -1.2820173502    0.2651366591
BR      2.9471905231   -1.8311738968   -1.1635470390
C       0.2235048115   -1.8812854290   -0.2213225216
BR     -2.4441890717   -2.4153206348    0.7312369347
C       0.3229579031   -3.3842358589   -0.4009576142
C       0.8428428769   -4.1012687683    0.8560197949
H       0.9730407596   -3.6015074253   -1.2328238487
H      -0.6458840966   -3.7763621807   -0.6637045145
H       1.8381483555   -3.7647776604    1.1165392399
H       0.8862006068   -5.1690993309    0.6776776314
H       0.1923355013   -3.9302730560    1.7041733265



In [192]:
# If optimization is NOT successfull 

if  opt:
	template = environment.get_template("gamess_input_template.inp")
	data = []
	for num, line in enumerate(lines):
		if "TOTAL NUMBER OF ATOMS" in line:
			n_atoms = int(line.split()[5])
		if "ALWAYS THE LAST POINT COMPUTED!" in line:
			start_coord_index = num + 4
			end_coord_index = start_coord_index + n_atoms
			for i in range(start_coord_index, end_coord_index):
				data.append(lines[i])
				
	with open("gamess_parameters.in", 'r') as f:
		lines = f.readlines()
	for line in lines:
		if "SCFTYP" in line:
			SCFTYP = line.split()[2]
		if "MULT" in line:
			MULT = line.split()[2]
		if "OPTTOL" in line:
			OPTTOL = line.split()[2]
		if "NSTEP" in line:
			NSTEP = line.split()[2]
		if "GBASIS" in line:
			GBASIS = line.split()[2]
		if "NGAUSS" in line:
			NGAUSS = line.split()[2]
		if "MOLNAME" in line:
			MOLNAME = line.split()[2]


	content = template.render(SCFTYP=SCFTYP, MULT=MULT, OPTTOL=OPTTOL,
			NSTEP=NSTEP, GBASIS=GBASIS, NGAUSS=NGAUSS, MOLNAME=MOLNAME, COORDINATES=data)
	print(content)
	
	# write into a file
	# filename = MOLNAME+'.inp'
	# with open(filename, mode="w", encoding="utf-8") as message:
	# 	message.write(content)

In [193]:
with open("cluster_parameters.in", 'r') as f:
    lines = f.readlines()
for line in lines:
    if "WALLTIME" in line:
        WALLTIME = line.split()[2]
    if "BIN" in line:
        BIN = line.split()[2]
    if "USER_EMAIL" in line:
        USER_EMAIL = line.split()[2]

template = environment.get_template("array_sub_template.sh")

content = template.render(WALLTIME=WALLTIME, BIN=BIN, USER_EMAIL=USER_EMAIL)
print(content)

#!/bin/sh
file_xyz=`basename $(ls *.xyz)`
file_in=`basename $(ls *.in)`

keyword_ntraj=`head -n 1 $file_in | awk '{ print $1 }'` 

if [[ "$keyword_ntraj" ]]; then
    n_traj=`head -n 1 $file_in | awk '{ print $2 }'`
else 
    n_traj=$((`head -n 1 $file_xyz`*25))
fi

molname=`basename $file_xyz .xyz`
walltime=02:00:00
bin=/storage/brno2/home/wrojasv/MassSpec/QC_Spectra_Predictions/local_run_test/bin_test
user_email=wrojasv@recetox.muni.cz 

echo 'submitted neutral MD job'
job1=$(qsub -N $molname -M $user_email -l walltime=$walltime -v "bin=$bin" $bin/neutral_run_md.pbs)

echo 'submitted prep production MD job'
job2=$(qsub -W depend=afterok:$job1 -N $molname -M $user_email -l walltime=$walltime -v "bin=$bin" $bin/prep_prod_run_ei_md.pbs)

echo 'submitted production run EI MD job'
job3=$(qsub -W depend=afterok:$job2 -N $molname -M $user_email -l walltime=$walltime -v "bin=$bin" -J 1-$n_traj $bin/prod_run_ei_md.pbs)
